In [50]:
import os
import time, csv
from datetime import datetime

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from faker import Faker

from textwrap import dedent
import os

########################################################################################################################
# Functions

BASE_DIR = os.path.dirname(os.path.abspath("_file_"))


def get_proxy_auth_extension(proxy):
    if proxy.strip() == '':
        return None
    cred, prox = proxy.split('@')
    PROXY_USER, PROXY_PASS = cred.split(':')
    PROXY_HOST, PROXY_PORT = prox.split(':')
    manifest_json = """
                    {
                        "version": "1.0.0",
                        "manifest_version": 2,
                        "name": "Chrome Proxy",
                        "permissions": [
                            "proxy",
                            "tabs",
                            "unlimitedStorage",
                            "storage",
                            "<all_urls>",
                            "webRequest",
                            "webRequestBlocking"
                        ],
                        "background": {
                            "scripts": ["background.js"]
                        },
                        "minimum_chrome_version":"22.0.0"
                    }
                    """

    background_js = """
                    var config = {
                            mode: "fixed_servers",
                            rules: {
                            singleProxy: {
                                scheme: "http",
                                host: "%s",
                                port: parseInt(%s)
                            },
                            bypassList: ["localhost"]
                            }
                        };

                    chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

                    function callbackFn(details) {
                        return {
                            authCredentials: {
                                username: "%s",
                                password: "%s"
                            }
                        };
                    }

                    chrome.webRequest.onAuthRequired.addListener(
                                callbackFn,
                                {urls: ["<all_urls>"]},
                                ['blocking']
                    );
                    """ % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)
    fn = os.path.join(BASE_DIR, 'proxy_ext')
    if not os.path.exists(fn):
        os.mkdir(fn)
    m_path = os.path.join(fn, "manifest.json")
    b_path = os.path.join(fn, "background.js")
    with open(m_path, 'w') as f:
        f.write(dedent(manifest_json))
    with open(b_path, 'w') as f:
        f.write(dedent(background_js))
    return fn


def get_file_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n')
    return data


def find_element_send_text(ele, text, clear=True):
    while True:
        try:
            input_field = driver.find_element(By.XPATH, ele)
            if clear:
                input_field.clear()
            input_field.send_keys(text)

            break
        except:
            time.sleep(0.1)


def specific_clicker(ele):
    while True:
        try:
            element = driver.find_element(By.XPATH, ele)
            webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

            break
        except Exception as e:
            # print(e)
            pass


def specific_clicker2(ele):
    try:
        element = driver.find_element(By.XPATH, ele)
        webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

    except Exception as e:
        # print(e)
        pass


def element_xpath_with_text(text):
    return f"//*[text()='{text}' or text()='{text.upper()}' or text()='{text.lower()}' or text()='{text.capitalize()}']"


def get_text(ele):
    while True:
        try:
            element = driver.find_elements(By.XPATH, ele)
            texts = [str(x.text) for x in element]
            return texts
        except Exception as e:
            # print(e)
            pass


def login_to_google_account(mail_address, mail_pwd, mail_recovery=False):
    find_element_send_text('//input[@type="email"]', mail_address)
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@type="email"]').send_keys(Keys.ENTER)

    find_element_send_text('//input[@type="password"]', mail_pwd)
    time.sleep(1)
    driver.find_element(By.XPATH, '//input[@type="password"]').send_keys(Keys.ENTER)


def business_details_fill(b_name, b_address, card_number, card_month, card_year, card_cvv):
    while True:
        try:
            iframe_element = driver.find_element(By.XPATH, '//iframe[@title="Enter payment info"]')
            driver.switch_to.frame(iframe_element)
            print("switched")
            break
        except:
            time.sleep(2)
    find_element_send_text('//input[@name="ORGANIZATION"]', b_name)
    find_element_send_text('//input[@name="ADDRESS_LINE_1"]', b_address)
    time.sleep(2)
    while True:
        try:
            driver.find_element(By.XPATH, '//input[@name="ADDRESS_LINE_1"]/..//div[@role="listbox"]/div').click()
            break
        except:
            time.sleep(3)

    find_element_send_text('//input[@name="cardnumber"]', card_number)
    find_element_send_text('//input[@name="ccmonth"]', card_month)
    find_element_send_text('//input[@name="ccyear"]', card_year)
    find_element_send_text('//input[@name="cvc"]', card_cvv)

    try:

        driver.find_element(By.XPATH, '//*[contains(@class, "message-checkbox")]').click()
    except:
        pass
    driver.switch_to.default_content()
    specific_clicker("//*[text()='Continue']")

def merchant_center_join(mail_address, mail_pwd,  b_name, b_address, card_number, card_month, card_year, card_cvv, mail_recovery=False):
    driver.get('https://www.google.com/retail/solutions/merchant-center/')
    # zoom out
    driver.execute_script("document.body.style.zoom='70%'")
    time.sleep(2)
    signup_link = driver.find_element(By.XPATH, "//h1/..//a[text()='Sign up for free']").get_attribute("href")
    driver.get(signup_link)

    login_to_google_account(mail_address, mail_pwd, mail_recovery)
    time.sleep(3)
    specific_clicker('(//*[@aria-label="No"])[1]')
    time.sleep(1.5)
    specific_clicker('(//*[@aria-label="No"])[2]')
    specific_clicker("//*[text()='Continue with Merchant Center Account creation']")
    specific_clicker("//*[text()='Continue to Merchant Center']")

    fake = Faker()
    random_company_name = fake.name() + " company"
    find_element_send_text('//input[@type="text"]', random_company_name)
    specific_clicker('//div[@aria-haspopup="listbox"]')
    time.sleep(1)
    specific_clicker("//span[text()='Italy']")
    specific_clicker("//*[text()='Continue to Merchant Center']")

    specific_clicker("//span[text()='Continue']")

    specific_clicker("//*[text()='Growth']")
    time.sleep(3)
    specific_clicker("//*[text()='Manage programs']")

    specific_clicker("//div[text()='Shopping ads']/../..//*[text()='Get started']")
    specific_clicker("//span[text()='Link to Google Ads']")

    specific_clicker("//*[text()='Create account & continue']")
    specific_clicker("//*[text()='Create & link account']")

    # Add billing Details
    business_details_fill(b_name, b_address, card_number, card_month, card_year, card_cvv)
    time.sleep(8)




def set_google_ads(keywords, website_url, headlines, descriptions):
    driver.get("https://ads.google.com/")
    specific_clicker(element_xpath_with_text("Sign in"))
    time.sleep(6)
    driver.get("https://ads.google.com/aw/overview")
    specific_clicker('//button[@aria-label="New campaign"]')
    specific_clicker("//span[text()='Website traffic']")
    specific_clicker("//span[text()='Search']")
    specific_clicker("//div[text()='Continue']")
    specific_clicker("//*[text()='Next']")
    specific_clicker("//*[text()='Italy']/../../div")
    specific_clicker("(//material-button//span[text()='Next'])[2]")
    # keywords

    find_element_send_text('//textarea', keywords)
    find_element_send_text('//input[@aria-label="Final URL"]', website_url)

    # headlines
    for index, headline in enumerate(headlines):
        try:
            driver.find_element(By.XPATH, f"(//span[text()='Headline']/../../input)[{index+1}]").send_keys(headline)
        except:
            pass


    # descriptions
    for index, description in enumerate(descriptions):
        try:
            driver.find_element(By.XPATH, f"(//section//textarea)[{index+1}]").send_keys(description)
        except:
            pass

    specific_clicker("//*[text()='Done']")
    specific_clicker("(//span[text()='Next'])[3]")

    # budget
    find_element_send_text('//input[@aria-label="Set your average daily budget for this campaign"]', "2.00")

    specific_clicker("(//span[text()='Next'])[4]")
    specific_clicker("(//span[text()='Publish campaign'])[5]")











In [2]:
options = uc.ChromeOptions()
# proxies_list = get_file_data('proxies.txt')
# my_proxy = random.choice(proxies_list)
# proxy = get_proxy_auth_extension(my_proxy)
# options.add_argument(f'--load-extension={proxy}')
driver = uc.Chrome(options=options)
driver.maximize_window()



In [3]:
merchant_center_join('KevinAnderson2543201@gmail.com', 'YCAVuC9FCf8')


KeyboardInterrupt



'https://merchants.google.com/signup/?hl=en&fmp=1&gfr_referral=true&mcsubid=us-en-web-g-mc-gfr'

In [51]:
set_google_ads("scarpe da uomo", "https://www.google.com", ["scarpe da uomo 1", "scarpe da uomo 2", "scarpe da uomo 3"], ["scarpe da uomo", "scarpe da uomo 2"])

In [14]:
# KevinSmith3876840@gmail.com:uDpqdUty:jo.ruiz.7x1oe@rambler.ru


KeyboardInterrupt



In [48]:
# zoom out 70%
driver.execute_script("document.body.style.zoom='70%'")